In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


After download en_core_web_lg, need to restart the runtime in google colab

In [0]:
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 827.9MB 105.4MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-cp36-none-any.whl size=829180944 sha256=e60b7257a9f2d8f37cca742a5242143616e1efbdabe5d1c189224686a9ef098a
  Stored in directory: /tmp/pip-ephem-wheel-cache-9orjjc99/wheels/2a/c1/a6/fc7a877b1efca9bc6a089d6f506f16d3868408f9ff89f8dbfc
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


## Basic Setup

In [0]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
import spacy
import os
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords

nlp = spacy.load('en_core_web_lg')  # load spaCy's built-in English models

vocabulary = list(nlp.vocab.strings)

## Similarity Detection

Load file

In [0]:
def load(file):
    with open(file, 'r', encoding='utf-8') as fr:
        text = fr.read()
    return nlp(text)

Load Corpus

In [0]:
def getCorpus(text_nlp, vocabulary=vocabulary): 
    corpus = list()
    for sent in text_nlp.sents:
        lemma = sent.lemma_.lower()
        ls = list()
        for token in sent:
            if not (nlp.vocab[lemma].is_stop or token.pos_ == 'PUNCT' or token.pos_ == 'SPACE'):
                ls.append(token.lemma_)
    #             vocabulary.add(token.lemma_)
        corpus.append(' '.join(ls))
    return corpus

Get TFIDF

In [0]:
def getTFIDF(corpus):
    pipe = Pipeline([('count', CountVectorizer(vocabulary=list(vocabulary))),
                 ('tfid', TfidfTransformer())]).fit(corpus)
    tfidf = pipe.transform(corpus)
    return tfidf

Basic version (three colors) of showing difference, should not be in final deliverables

In [0]:
def diff(sim, text_ls1, text_ls2, prefix_text1='', prefix_text2=''):
    '''
    compare text_ls1 with text_ls2
    mark different levels of similarity
    '''
    n,m = sim.shape
    # find the most similar sentence for each sentence and get the highest similarity score
    score = np.max(sim, axis = -1)
    indices = np.argmax(sim, axis = -1)
    # print out the result in console
    print(f"\033[1;30;43m small change \033[0m  \033[1;30;42m moderate change \033[0m \033[1;30;45m big change \033[0m")
    for i in range(n):
        j = indices[i]

        if sim[i, j] >=0.99:
            print(f"\033[0;30;31m [{prefix_text1}] \033[0m {text_ls1[i]} ")
            print(f"\033[0;30;31m [{prefix_text2}] \033[0m {text_ls2[j]}\n")
        if  0.8 <= sim[i,j] < 0.99:
            print(f"\033[0;30;31m [{prefix_text1}] \033[0m \033[1;30;43m {text_ls1[i]} \033[0m")
            print(f"\033[0;30;31m [{prefix_text2}] \033[0m {text_ls2[j]}\n")
        elif  0.5 <= sim[i,j] < 0.8:
            print(f"\033[0;30;31m [{prefix_text1}] \033[0m \033[1;30;42m {text_ls1[i]} \033[0m")
            print(f"\033[0;30;31m [{prefix_text2}] \033[0m {text_ls2[j]}\n")
        elif sim[i,j] < 0.5:
            print(f"\033[0;30;31m [{prefix_text1}] \033[0m \033[1;30;45m {text_ls1[i]} \033[0m")
            print(f"\033[0;30;31m [{prefix_text2}] \033[0m {text_ls2[j]}\n")

Calculate Cosine Similarity and Call Visualization

In [0]:
def diff_analysis(file1, file2, file1_suffix='', file2_suffix=''):
    file1_nlp = load(file1)
    file2_nlp = load(file2)

    file1_ls = list([x.text.strip() for x in file1_nlp.sents])
    file2_ls = list([x.text.strip() for x in file2_nlp.sents])
    
    # create dictionary for visualization
    dic_1 = {}
    dic_2 = {}
    for i in range(len(file1_ls)):
        dic_1[i] = file1_ls[i]
    dic_1[-1] = "-----"
    for i in range(len(file2_ls)):
        dic_2[i] = file2_ls[i]
    dic_2[-1] = "-----"
    
    # get corpus
    file1_corpus = getCorpus(file1_nlp)
    file2_corpus = getCorpus(file2_nlp)
    # get tfidf
    file1_tfidf = getTFIDF(file1_corpus)
    file2_tfidf = getTFIDF(file2_corpus)

    # get similarity
    sim = cosine_similarity(file1_tfidf, file2_tfidf, dense_output=True)

    # just show the diff in console
    # comment out in production
    # diff(sim, file1_ls, file2_ls, file1_suffix, file2_suffix)
    
    # call similarity detection visualization
    pairs, addition, deletion, dic_curr, dic_last, score = matrix_mani(sim)
    write_html(pairs, deletion, dic_1, dic_2, file1_suffix, file2_suffix, score)
    
    # call sentiment analysis visualization
    dic_sent = build_dic_sent(file1_ls, file2_ls)
    sent_html(dic_sent, dic_curr, dic_last, dic_1, dic_2, file1_suffix, file2_suffix)
    
    # return sim

## Similarity Detection Visualization

Create list of pairs from similarity matrix

In [0]:
def matrix_mani(sim):
    '''
    Input matrix is pandas DataFrame
    Column is the sentence number of newer 10K, starting from 0
    Row is the sentence number of older 10K, starting from 0
    Data for each individual cell is the cosine similarity between two sentences marked by the column and row.
    
    Output are three python list:
    First list contains pairs of numbers as a small list, in each small list, the first number is the sentence number
    in the newer 10K, second number is the sentence number corresponding in the older 10K, and -1 stands for there is no match
    Second list contains several numbers which indicate in the newer 10K which sentence is added
    Third list contains several numbers which indicate form the older 10K which sentence is deleted
    '''
    matrix = pd.DataFrame(sim, index = range(0, len(sim)), columns=range(0,len(sim[0])))
    matrix = matrix.transpose()

    rows, columns = matrix.shape
    # find pair of sentence
    current_10k = []
    # build up dictionary for collision
    corr_sent = {}
    for column in range(columns):
        current_sent = column
        try:
            corres_sent = matrix[str(current_sent)].idxmax()
            value = matrix[str(current_sent)].max()
        except:
            corres_sent = matrix[int(current_sent)].idxmax()
            value = matrix[int(current_sent)].max()
        # append pair to the list
        current_10k.append([int(current_sent), int(corres_sent), value])
        # build up dictionary
        if corres_sent not in corr_sent:
            corr_sent[corres_sent] = []
        corr_sent[corres_sent].append((current_sent, value))
    addition = []
    deletion = []
    # go through the dictionary build up to detect collsion
    for item in corr_sent:
        # if there is collision: ie more than 1 sentences from the newer 10K is mapped to the same sentence from the older 10K
        # Choose the sentence from newer 10K with highest similarity, and the sentences with lower similarity will be recognized as
        # new sentence
        # there are extreme situation where a specific sentence in the newer 10K are quite similar to two of the older sentence,
        # this code fail to detect that.
        if len(corr_sent[item]) > 1:
            pairs = corr_sent[item]
            values = [pair[1] for pair in pairs]
            sents = [pair[0] for pair in pairs]
            max_value = max(values)
            false_pos = [i for i, value in enumerate(values) if value != max_value]
            false_curr = [sents[pos] for pos in false_pos]
            for item in false_curr:
                current_10k[item][1] = -1
                current_10k[item][2] = 0
                addition.append(item)
    addition.sort()
    # detect deletion, which is the key not in the dictionary (sentence presented in the older 10K but no match in the newer document)
    for row in range(rows):
        if row not in corr_sent:
            deletion.append(row)
            
    # build up matching dictionary for sentiment analysis visualization
    dic_curr = {}
    dic_last = {}
    actual = 0
    potential = 0
    for item in current_10k:
        dic_curr[item[0]] = item[1]
        dic_last[item[1]] = item[0]
        actual += item[2]
        potential += 1
    for item in addition:
        dic_curr[item] = -1
        potential += 1
    for item in deletion:
        dic_last[item] = -1
        potential += 1
    return current_10k, addition, deletion, dic_curr, dic_last, actual/potential

Create HTML file

In [0]:
def write_html(pairs, deletion, dic_1, dic_2, file1_suffix, file2_suffix, score):
    file1_suffix, *_ = file1_suffix.split(".")
    file2_suffix, *_ = file2_suffix.split(".")
    
    dic_d = {}
    for item in deletion:
        dic_d[item-1] = item
        
    nums = [0.98, 0.8, 0.6, 0.4, 0, -1]
    colors = ["#FFFFFF","#FF9F9F","#FF7E7E","#FF4848","#FF0000","#B31717"]
    colors = list(zip(nums, colors))
#     score = str(round(score * 100, 4) + "%"
    htmlPage = "<html><body><table border='1'>"
    htmlPage += "<tr><th>Overall Similarity</th>"
    htmlPage += "<th>" + str(round(score*100, 4)) + "%</th></tr>"
    htmlPage += "<tr><th>" + file1_suffix + "</th>"
    htmlPage += "<th>" + file2_suffix + "</th></tr>"

    for pair in pairs:        
        for color in colors:
            if pair[2] > color[0]:
                use_color = color[1]
                break        
        htmlPage += "<tr><td width='50%' valign='top'><span style='background-color: " + use_color + "'>" + dic_1[pair[0]] + "</span></td><td valign='top'>" + dic_2[pair[1]] + "</td></tr>"
        if pair[1] in dic_d:
            use_color = "#B31717"
            htmlPage += "<tr><td width='50%' valign='top'><span style='background-color: " + use_color + "'>" + dic_1[-1] + "</span></td><td valign='top'>" + dic_2[dic_d[pair[1]]] + "</td></tr>"
    htmlPage +="</table></body></html>"        
    fileName = "Difference_" + str(file1_suffix) + "_" + str(file2_suffix) + ".html"
    with open(fileName, "w", encoding='utf-8') as f: 
        f.write(htmlPage) 
    f.close()

## Sentiment Analysis Visualization

Lammentize word function

In [0]:
def lemmatize_words(words):
    """
    Lemmatize words 

    Parameters
    ----------
    words : list of str
        List of words

    Returns
    -------
    lemmatized_words : list of str
        List of lemmatized words
    """

    lemmatized_words = [WordNetLemmatizer().lemmatize(w, pos='v') for w in words]

    return lemmatized_words

Build a dictionary based on LoughranMcDonald_MasterDictionary_2018.csv

In [0]:
def build_dic():
    # load sentiment vocab
    sentiment_df = pd.read_csv('LoughranMcDonald_MasterDictionary_2018.csv') # From University of Notre Dame

    sentiments = ['negative', 'positive', 'uncertainty', 'litigious', 'constraining', 'interesting']

    sentiment_df.columns = [column.lower() for column in sentiment_df.columns] # Lowercase the columns for ease of use

    # Remove unused information
    sentiment_df = sentiment_df[sentiments + ['word']]
    sentiment_df[sentiments] = sentiment_df[sentiments].astype(bool)
    sentiment_df = sentiment_df[(sentiment_df[sentiments]).any(1)]

    # Apply the same preprocessing to these words as the 10-k words
    sentiment_df['word'] = lemmatize_words(sentiment_df['word'].str.lower())
    sentiment_df = sentiment_df.drop_duplicates('word')

    # create a dictionary with words and sentiments
    dic_sent = {}
    for index, row in sentiment_df.iterrows():
        dic_sent[row['word']] = []
        dic_sent[row['word']].append([row['negative'], row['positive'], row['uncertainty'], row['litigious'], row['constraining'], row['interesting']])
        dic_sent[row['word']].append([])
        dic_sent[row['word']].append([])
    return dic_sent

Filter specific sentences using sentiment dictionary

In [0]:
def build_dic_sent(file1, file2):
    dic_sent = build_dic()
    for i in range(len(file1)):
        words = file1[i].split(" ")
        for word in words:
            if word in dic_sent:
                dic_sent[word][1].append(i)
    for i in range(len(file2)):
        words = file2[i].split(" ")
        for word in words:
            if word in dic_sent:
                dic_sent[word][2].append(i)
    return dic_sent

Build sentiment html file

In [0]:
def sent_html(dic_sent, dic_curr, dic_last, dic_1, dic_2, file1_suffix, file2_suffix):
    sentiments = ['Negative', 'Positive', 'Uncertainty', 'Litigious', 'Constraining', 'Interesting']
    file1_suffix, *_ = file1_suffix.split(".")
    file2_suffix, *_ = file2_suffix.split(".")
    # create HTML file
    htmlPage = "<html><body><table border='1'>"
    htmlPage += "<tr><th>Action</th>"
    htmlPage += "<th>" + file1_suffix + "</th>"
    htmlPage += "<th>" + file2_suffix + "</th></tr>"
    # in the dictionary, if a word in the dictionary has representation (sentences) in either of the two reports, move on
    for key, value in dic_sent.items():
        size1 = len(value[1])
        size2 = len(value[2])
        if size1 != 0 or size2 != 0:
            # detect from the two reports that whether the word shows up reflect a change
            lis_curr = []
            curr = {}
            last = {}
            for a in value[1]:
                curr[a] = 1
            for b in value[2]:
                last[b] = 1
            # if the one of the sentences contains no words, the print out the pair
            # print the category of the change
            for item in value[1]:
                if dic_curr[item] in last:
                    continue
                elif dic_curr[item] == -1:
                    lis_curr.append(["Sentence Added", item, -1])
                else:
                    lis_curr.append(["Key Word Added", item, dic_curr[item]])
            for item in value[2]:
                if dic_last[item] in curr:
                    continue
                elif dic_last[item] == -1:
                    lis_curr.append(["Sentense Deleted", -1, item])
                else:
                    lis_curr.append(["Key Word Deleted", dic_last[item], item])
            # actual printing of the sentences
            if len(lis_curr) != 0:
                sentiment_zip = (list(zip(sentiments, value[0])))
                sentiment_str = ""
                for j in range(5):
                    sentiment_str += sentiment_zip[j][0] + ": " + str(sentiment_zip[j][1]) + ", "
                sentiment_str += sentiment_zip[5][0] + ": " + str(sentiment_zip[5][1])
                htmlPage += "<tr><td width='10%' valign='top'> </td><td width='45%' valign='top'><span style='background-color:#ffff00'>" + key + "</td><td width='45%' valign='top'><span style='background-color:#ffff00'>" + sentiment_str + "</td></tr>"
                for item in lis_curr:
                    htmlPage += "<tr><td width='10%' valign='top'>" + item[0] + "</td><td width='45%' valign='top'>" + dic_1[item[1]] + "</td><td width='45%' valign='top'>" + dic_2[item[2]] + "</td></tr>"
    htmlPage +="</table></body></html>"
    # labeling filename
    fileName = "Sentiment_" + str(file1_suffix) + "_" + str(file2_suffix) + ".html"
    with open(fileName, "w", encoding='utf-8') as f: 
        f.write(htmlPage) 
    f.close()

## Leader Function

Leader function that calls other functions

In [0]:
def diff_files(file_dir, ls):
    '''
    input:
    file_dir: directory of the 10k or 10Q files
    ls: list of 10k or 10Q file names, ordered by time decreasely,
    eg: ls = ['APPLE COMPUTER INC_19_10k.txt','APPLE COMPUTER INC_18_10k.txt',
              'APPLE COMPUTER INC_17_10k.txt'], compares apple 19 10k to 18 10k, 
              then compares 18 10k to 17 10k

    output:
    similarity score list
    '''
    sim_ls = []
    for i in range(len(ls)-1):
        file1 = os.path.join(file_dir, ls[i])
        file2 = os.path.join(file_dir, ls[i+1])
        diff_analysis(file1, file2, ls[i], ls[i+1])

## Testing

In [0]:
'''
Common function:
get files from a directory and organize the files by company names,
then detect similarity by companys in time reversed order.
'''
file_dir = r'/content/drive/My Drive/Capstone_file/'
ls = [f for f in os.listdir(file_dir) if '.txt' in f]
d  = dict()
for file in ls:
    company = file.split('_')[0]
    d[company] = d.get(company, []) + [file]


for company in d:
    ls = sorted(d[company])[::-1]
    print(ls)
    # diff_files(file_dir, ls)

In [0]:
# diff_analysis(r'/Users/dkxsj/PycharmProjects/ISM/PNC-2019', 
#               r'/Users/dkxsj/PycharmProjects/ISM/PNC-2018',
#              'PNC-2019-10K',
#              'PNC-2018-10K')

Test over 10Ks

In [0]:
file_dir = r'/content/drive/My Drive/Capstone_file/'
ls_10k = ['aapl_2019_10k.txt', 
          'aapl_2018_10k.txt',
          'aapl_2017_10k.txt']
diff_files(file_dir, ls_10k)

Test over 10Qs

In [0]:
file_dir = r'/content/drive/My Drive/Capstone_file/'
ls_10Q = ['aapl_2019_3_10q.txt',
          'aapl_2019_2_10q.txt',
          'aapl_2019_1_10q.txt',
          'aapl_2018_3_10q.txt',
          'aapl_2018_2_10q.txt',
          'aapl_2018_1_10q.txt']

diff_files(file_dir, ls_10Q)

 small change    moderate change   big change 
 [aapl_2019_3_10q.txt]  >Item 1A. 
 [aapl_2019_2_10q.txt]  >Item 1A.

 [aapl_2019_3_10q.txt]  Risk Factors 
 [aapl_2019_2_10q.txt]  Risk Factors

 [aapl_2019_3_10q.txt]   The following description of risk factors includes any material changes to, and supersedes the description of, risk factors associated with the Company’s business previously disclosed in Part I, Item 1A of the 
2018
 Form 10-K and in Part II, Item 1A of the Forms 10-Q for the quarters ended December 29, 2018 and March 30, 2019, in each case under the heading “Risk Factors.” 
 [aapl_2019_2_10q.txt]  The following description of risk factors includes any material changes to, and supersedes the description of, risk factors associated with the Company’s business previously disclosed in Part I, Item 1A of the 
2018
 Form 10-K and in Part II, Item 1A of the Form 10-Q for the quarter ended December 29, 2018, in each case under the heading “Risk Factors.”

 [aapl_2019_3_10q.txt] 

[array([[1.        , 0.        , 0.3800915 , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 1.        , 0.30934519, ..., 0.        , 0.        ,
         0.        ],
        [0.37079469, 0.30186567, 0.95610298, ..., 0.11979917, 0.04989866,
         0.21113925],
        ...,
        [0.        , 0.        , 0.12122922, ..., 0.99999983, 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.74628745,
         0.        ],
        [0.        , 0.        , 0.28852059, ..., 0.        , 0.        ,
         1.        ]]),
 array([[1.        , 0.        , 0.26429066, ..., 0.        , 0.        ,
         0.        ],
        [0.        , 1.        , 0.43018833, ..., 0.        , 0.        ,
         0.        ],
        [0.3800915 , 0.30933921, 0.87251728, ..., 0.12120875, 0.        ,
         0.28852059],
        ...,
        [0.        , 0.        , 0.11782963, ..., 0.99999973, 0.        ,
         0.        ],
        [0. 